In [93]:
# mypoll/ORM_Exam.ipynb

## Jupyter Lab에서 Django shell을 실행하기 위한 설정
import os
import django
# 환경변수로 config/settings.py의 위치를 설정
os.environ['DJANGO_SETTINGS_MODULE'] = "config.settings"
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = "true"

django.setup()

In [94]:
# 조회 테스트
from polls.models import Question, Choice

q = Question.objects.all()
q

<QuerySet [<Question: 1. 좋아하는 색깔은 무엇입니까?>, <Question: 2. 좋아하는 동물은 무엇입니까?>, <Question: 3. 싫어하는 색깔은 고르세요.>, <Question: 4. 여행으로 가고 싶은 나라를 선택해주세요.>, <Question: 5. 배우고 싶은 언어는 무엇입니까?>]>

0

# 조회
- ModelClass.objects -> Model Manager를 반환.
- ModelManager: SQL작업을 할 수있는 메소드들을 제공하는 객체

## 조회메소드
- `all()`: 전체 조회
- `filter()`, `exclude()`: 조건으로 조회(where절)
- `get()`: 조회결과가 하나인 조건으로 조회(PK로 조회)

## 조회결과
- `QuerySet` 객체: 조회결과가 여러개일때 QuerySet에 모아서 반환.
  - 조회결과를 바탕으로 추가 DB 작업을 진행할 수 있는 메소드들을 제공.
  - 개별 데이터는 Model 객체에 담아서 반환.
- Model 객체: 조회결과가 하나(`get()`) 일 때 

In [3]:
from polls.models import Question, Choice

In [4]:
model_manager = Question.objects
type(model_manager)

django.db.models.manager.Manager

In [7]:
result = model_manager.all()
print("조회한 데이터개수:", len(result))
print("all()로 실행된 SQL문을 조회")
print(result.query)

조회한 데이터개수: 4
all()로 실행된 SQL문을 조회
SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question"


In [10]:
print(type(result))
result

<class 'django.db.models.query.QuerySet'>


<QuerySet [<Question: 1. 좋아하는 색깔은 무엇입니까?>, <Question: 2. 좋아하는 동물은 무엇입니까?>, <Question: 3. 싫어하는 색깔은 고르세요.>, <Question: 4. 여행가고 싶은 나라는?>]>

In [13]:
# QuerySet -> Iterable
for q in result:
    # print(type(q))
    print(q, q.pub_date)

1. 좋아하는 색깔은 무엇입니까? 2025-07-07 05:17:02.657964+00:00
2. 좋아하는 동물은 무엇입니까? 2025-07-07 05:17:27.441139+00:00
3. 싫어하는 색깔은 고르세요. 2025-07-07 05:17:41.937410+00:00
4. 여행가고 싶은 나라는? 2025-07-07 06:15:03.568627+00:00


In [19]:
# QuerySet -> Subscriptable
q = result[1]
print(q.id, q.pk)
print(q.question_text, q.pub_date)
print(type(q.pk), type(q.question_text), type(q.pub_date))

2 2
좋아하는 동물은 무엇입니까? 2025-07-07 05:17:27.441139+00:00
<class 'int'> <class 'str'> <class 'datetime.datetime'>


In [20]:
# QuerySet - 첫번째, 마지막 index값 조회
q_s = result.first()
q_e = result.last()
q_s.pk, q_e.pk

(1, 4)

In [24]:
# slicing -> 결과: list
s_result = result[:3]
print(type(s_result))
s_result

<class 'list'>


[<Question: 1. 좋아하는 색깔은 무엇입니까?>,
 <Question: 2. 좋아하는 동물은 무엇입니까?>,
 <Question: 3. 싫어하는 색깔은 고르세요.>]

In [27]:
# 음수 indexing은 지원하지 않는다.
# result[-2] 

In [32]:
## QuerySet을 이용해서 정렬 (order by) 
##  - QS.order_by("기준Field명"): ASC, QS.order_by("-기준Field명"): DESC
# result.order_by("question_text")
result.order_by("-question_text")

<QuerySet [<Question: 1. 좋아하는 색깔은 무엇입니까?>, <Question: 2. 좋아하는 동물은 무엇입니까?>, <Question: 4. 여행가고 싶은 나라는?>, <Question: 3. 싫어하는 색깔은 고르세요.>]>

In [38]:
# Choice에 모든 데이터를 조회 -> choice_text 기준으로 정렬
results = Choice.objects.all().order_by("choice_text")
for c in results:
    print(c.pk, c.choice_text, c.votes)

10 개 0
6 검정 0
11 고양이 5
9 곰 0
3 노랑 0
4 보라 0
7 보라 0
1 빨강 10
5 빨강 0
8 주황 0
2 파랑 0
12 호랑이 3


In [41]:
# order by choice_text, votes desc
results = Choice.objects.all().order_by("choice_text", "-votes")
for c in results:
    print(c.pk, c.choice_text, c.votes)

10 개 0
6 검정 0
11 고양이 5
9 곰 0
3 노랑 0
7 보라 20
4 보라 0
5 빨강 10
1 빨강 5
8 주황 0
2 파랑 0
12 호랑이 3


In [42]:
print(results.query)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" ORDER BY "polls_choice"."choice_text" ASC, "polls_choice"."votes" DESC


### Where 절
- filter()
  - 조회조건이 True인 행들을 조회 -> QuerySet을 반환
- exclude()
  - 조회조건이 False인 행들을 조회 -> QuerySet을 반환
- get()
  - 조회조건이 True인 행이 1개일 때 조회.-> Model에 결과를 담아서 반환.
  - 조회결과가 2행 이상이거나 없을 경우 Exception 발생.
- **조회조건**
  - `Field이름__비교연산자 = 비교할 값`

In [ ]:
# pk 조회 -> 결과 1행 | 0행
# 동등 비교: field명 = 값
result = Question.objects.get(pk=1)    # where id = 1. 조회결과가 1개
result = Question.objects.filter(pk=1) # where id = 1. 조회결과가 0개 이상
result = Question.objects.exclude(pk=1)# where not id = 1

print(type(result))
print(result)



<class 'django.db.models.query.QuerySet'>
<QuerySet [<Question: 2. 좋아하는 동물은 무엇입니까?>, <Question: 3. 싫어하는 색깔은 고르세요.>, <Question: 4. 여행가고 싶은 나라는?>]>


In [54]:
# 비교 연산 
result = Choice.objects.filter(pk__lt=5) # where pk < 5
result = Choice.objects.filter(pk__lte=5) # where pk <= 5
result = Choice.objects.filter(pk__gt=5) # where pk > 5
result = Choice.objects.filter(pk__gte=5) # where pk >= 5
result = Choice.objects.filter(pk=5) # where pk = 5

for r in result:
    print(r)

5. 빨강


In [ ]:
# 문자열 부분일치 - like ( xxx를 포함, xxx로 시작, xxx로 끝)
result = Question.objects.filter(question_text__contains='색깔')
# question_text like '%색깔%'
result = Question.objects.filter(question_text__endswith='무엇입니까?')
# question_text like "%무엇입니까?"
result = Question.objects.filter(question_text__startswith="싫어하는")
# question_text like "싫어하는%"

print(result.query)
for r in result:
    print(r)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question" WHERE "polls_question"."question_text" LIKE 싫어하는% ESCAPE '\'
3. 싫어하는 색깔은 고르세요.


In [64]:
# in  연산
result = Choice.objects.filter(pk__in=[1, 10, 3]) # pk in (1, 10, 3)
result = Choice.objects.exclude(pk__in=[1, 10, 3]) # pk not in (1, 10, 3)

print(result.query)
for r in result:
    print(r)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" WHERE NOT ("polls_choice"."id" IN (1, 10, 3))
2. 파랑
4. 보라
5. 빨강
6. 검정
7. 보라
8. 주황
9. 곰
11. 고양이
12. 호랑이


In [66]:
# between
result = Choice.objects.filter(pk__range=[4, 8])
# pk between 4 and 8
result = Choice.objects.exclude(pk__range=[4, 8])
# pk not between 4 and 8

print(result.query)
for r in result:
    print(r)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" WHERE NOT ("polls_choice"."id" BETWEEN 4 AND 8)
1. 빨강
2. 파랑
3. 노랑
9. 곰
10. 개
11. 고양이
12. 호랑이


#### where 절의 and, or
- `and`: 조건들을 나열
- 각 조건을 `Q()` 함수에 넣고 `|` 로 연결.

In [ ]:
result = Question.objects.filter(
    question_text__endswith='무엇입니까?',
    pk__gte=2
)

print(result.query)
for r in result:
    print(r)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question" WHERE ("polls_question"."id" >= 2 AND "polls_question"."question_text" LIKE %무엇입니까? ESCAPE '\')
2. 좋아하는 동물은 무엇입니까?


In [ ]:
~Q(question_text__endswith='무엇입니까?')   # | Q(pk__gte=3)

<Q: (OR: ('question_text__endswith', '무엇입니까?'), ('pk__gte', 3))>

In [3]:
from django.db.models import Q
result = Question.objects.filter(
    Q(question_text__endswith='무엇입니까?') | Q(pk__gte=3)
)

print(result.query)
for r in result:
    print(r)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question" WHERE ("polls_question"."question_text" LIKE %무엇입니까? ESCAPE '\' OR "polls_question"."id" >= 3)
1. 좋아하는 색깔은 무엇입니까?
2. 좋아하는 동물은 무엇입니까?
3. 싫어하는 색깔은 고르세요.
4. 여행가고 싶은 나라는?


In [5]:
# ~Q() : Not
result = Question.objects.filter(
    ~Q(question_text__endswith='무엇입니까?') | Q(pk__gte=4)
)

print(result.query)
for r in result:
    print(r)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question" WHERE (NOT ("polls_question"."question_text" LIKE %무엇입니까? ESCAPE '\') OR "polls_question"."id" >= 4)
3. 싫어하는 색깔은 고르세요.
4. 여행가고 싶은 나라는?


In [7]:
result = Question.objects.filter(
    ~Q(question_text__endswith='무엇입니까?'), pk__lte=3
)

print(result.query)
for r in result:
    print(r)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question" WHERE (NOT ("polls_question"."question_text" LIKE %무엇입니까? ESCAPE '\') AND "polls_question"."id" <= 1)


#### 조회할 컬럼들을 선택.
- `values(Field명, ...)`
- 개별 조회 결과을 Dictionary로 반환.

In [85]:
result = Question.objects.all().values("pk", "question_text")
result = Choice.objects.filter(pk__lt = 5).values('pk', 'votes')
# select pk, votes from choice where pk < 5
print(result.query)
result
# for r in result:
#     print(type(r), r['pk'], r['question_text'])

SELECT "polls_choice"."id" AS "pk", "polls_choice"."votes" AS "votes" FROM "polls_choice" WHERE "polls_choice"."id" < 5


<QuerySet [{'pk': 1, 'votes': 5}, {'pk': 2, 'votes': 0}, {'pk': 3, 'votes': 0}, {'pk': 4, 'votes': 0}]>

### 집계함수
- `aggregate(집계함수(집계기준field명), 집계함수(field명), ...)`
  - `select avg(salary), count(comm_pct), max(salary) from emp`
- **groupby**
  - values("groupby기준 컬럼").annotate(집계함수)

In [86]:
from django.db.models import (
    Count, # 값의 개수(null 제외한)
    Sum,   # 합계
    Avg,   # 평균
    Min,   # 최소
    Max,   # 최대값
    StdDev, # 표준편차
    Variance # 분산값
)

In [92]:
result = Choice.objects.aggregate(
    Count("votes"),
    Avg("votes"),
    Max("pk"),
    Min("pk"),
    Sum("votes"),
    StdDev("votes"),
    Variance("votes")

)

print(result)
# 반환: dictionary
# default key: field명__집계명

{'votes__count': 12, 'votes__avg': 3.5833333333333335, 'pk__max': 12, 'pk__min': 1, 'votes__sum': 43, 'votes__stddev': 5.808877305947816, 'votes__variance': 33.74305555555556}


In [93]:
result = Choice.objects.aggregate(
    cnt=Count("votes"),
    min=Min("votes"),
    max=Max("votes")
)
result

{'cnt': 12, 'min': 0, 'max': 20}

In [99]:
# 집계 결과를 연산
# 변수명 = (집계함수 - 집계함수)
Choice.objects.aggregate(min_max_diff = Max("pk") - Min("pk"))

{'min_max_diff': 11}

In [101]:
######### groupby + 집계
result = Choice.objects.values("question").annotate(
    min=Min("votes"),
    max=Max("votes")
)

In [102]:
result

<QuerySet [{'question': 1, 'min': 0, 'max': 5}, {'question': 2, 'min': 0, 'max': 5}, {'question': 3, 'min': 0, 'max': 20}]>

### JOIN
- 자식 테이블(모델) 기준으로 부모 테이블(모델) 데이터 조회.
- `자식모델객체.FK_Field`

In [14]:
# Choice(자식) --> Question(부모)
c1 = Choice.objects.get(pk=1)
c1.pk, c1.id, c1.choice_text, c1.votes, c1.question

(1, 1, '빨강', 5, <Question: 1. 좋아하는 색깔은 무엇입니까?>)

In [17]:
# c1이 참조하는 question의 정보
c1.question.pk, c1.question.question_text, c1.question.pub_date

(1,
 '좋아하는 색깔은 무엇입니까?',
 datetime.datetime(2025, 7, 7, 5, 17, 2, 657964, tzinfo=datetime.timezone.utc))

In [22]:
result_list = Choice.objects.filter(pk__lte=5)
# 조회한 choice들의 질문 - 보기
for result in result_list:
    print(f"질문:{result.question.question_text}, 조회한 보기:{result.choice_text}")

질문:좋아하는 색깔은 무엇입니까?, 조회한 보기:빨강
질문:좋아하는 색깔은 무엇입니까?, 조회한 보기:파랑
질문:좋아하는 색깔은 무엇입니까?, 조회한 보기:노랑
질문:좋아하는 색깔은 무엇입니까?, 조회한 보기:보라
질문:싫어하는 색깔은 고르세요., 조회한 보기:빨강


- 부모테이블(모델) 기준으로 자식테이블의 데이터(모델)를 조회
  - `부모모델객체.자식모델클래스이름(소문자)_set`을 통해서 부모객체를 참조하는 자식 데이터들을 **조회**할 수 있다.

In [24]:
q1 = Question.objects.get(pk=1)
q1
print(q1.pk, q1.question_text, q1.pub_date)

1 좋아하는 색깔은 무엇입니까? 2025-07-07 05:17:02.657964+00:00


In [27]:
# RelatedManager 
#-> 부모객체(q1)와 관련있는 자식의 데이터들 안에서만 조회할 수있는 Model Manager
q1.choice_set

choice_list = q1.choice_set.all()
choice_list


<QuerySet [<Choice: 1. 빨강>, <Choice: 2. 파랑>, <Choice: 3. 노랑>, <Choice: 4. 보라>]>

In [29]:
print("질문:", q1.question_text)
print("보기")
for c in choice_list:
    print(f"{c.pk}. {c.choice_text}, {c.votes}")

질문: 좋아하는 색깔은 무엇입니까?
보기
1. 빨강, 5
2. 파랑, 0
3. 노랑, 0
4. 보라, 0


In [33]:
#  전체 질문을 조회하고 그것에 대해서 위 형식으로 출력.
q_list = Question.objects.all()
for q in q_list:
    print(f"{q.pk}. {q.question_text}")
    c_list = q.choice_set.all()
    for idx, c in enumerate(c_list, start=1):
        print(f"\t{idx}. {c.choice_text} - {c.votes}")

1. 좋아하는 색깔은 무엇입니까?
	1. 빨강 - 5
	2. 파랑 - 0
	3. 노랑 - 0
	4. 보라 - 0
2. 좋아하는 동물은 무엇입니까?
	1. 곰 - 0
	2. 개 - 0
	3. 고양이 - 5
	4. 호랑이 - 3
3. 싫어하는 색깔은 고르세요.
	1. 빨강 - 10
	2. 검정 - 0
	3. 보라 - 20
	4. 주황 - 0
4. 여행가고 싶은 나라는?


# insert / update
- `모델객체.save()`
- 모델객체의 pk 가 DB에 없으면 insert, 있으면 update

In [36]:
# insert
## question: pk-id : 자동증가 정수(생략), pub_date: insert할 때 일시를 추가(생략)
new_q = Question(question_text="배우고 싶은 언어는 무엇입니까?")
print(new_q.question_text)
print(new_q.pk, new_q.pub_date)

배우고 싶은 언어는 무엇입니까?
None None


In [37]:
# 저장 - pk: None (DB에 없는 데이터) ==> insert
new_q.save()

In [38]:
# insert 후에 자동저장되는 값들(pk, pub_date)이 모델객체의 field에 저장된다.
print(new_q.pk)
print(new_q.pub_date)

5
2025-07-08 02:22:32.175623+00:00


In [44]:
# update
q = Question.objects.get(pk=4)
q.question_text = "여행으로 가고 싶은 나라를 선택해주세요."

In [46]:
# q.pk = 4 -> DB에 있는 pk => update
q.save() 

In [47]:
for q in Question.objects.all():
    print(q)

1. 좋아하는 색깔은 무엇입니까?
2. 좋아하는 동물은 무엇입니까?
3. 싫어하는 색깔은 고르세요.
4. 여행으로 가고 싶은 나라를 선택해주세요.
5. 배우고 싶은 언어는 무엇입니까?


In [ ]:
# pk=4인 질문의 보기(choice)를 추가
q = Question.objects.get(pk=4)

In [59]:
# FK field -> 부모 Model 객체 (참조 Field값만 instance변수로 가지면 된다.)
c = Choice(choice_text="미국", question=q)
c.save()

In [61]:
c = Choice(choice_text="일본", votes=10, question=q)
c.save()

In [ ]:
qq = Question(pk=4)  # 부모 모델 객체는 PK만 있으면 된다.
# print(qq.pk, qq.question_text)
c = Choice(choice_text="영국", votes=20, question=qq)
c.save()

In [67]:
for c in q.choice_set.all():
    print(c.pk, c.choice_text, c.votes)

13 미국 0
14 일본 10
15 영국 20


# delete
- `모델객체.delete()`
  - 모델객체의 pk의 data를 DB에 삭제.


In [69]:
# delete from choice where id=15
c15 = Choice.objects.get(pk=15)
c15.delete()

(1, {'polls.Choice': 1})

In [72]:
# choice에서 id가 10 이상인 값들을 삭제
## delete from choice where id >= 10
del_c = Choice.objects.filter(pk__gte=10)
for c in del_c:
    c.delete()

In [73]:
for c in Choice.objects.all():
    print(c)

1. 빨강
2. 파랑
3. 노랑
4. 보라
5. 빨강
6. 검정
7. 보라
8. 주황
9. 곰


# 직접 SQL문을 실행

In [79]:
result = Question.objects.raw("select * from polls_question")

for r in result:
    print(r)

<RawQuerySet: select * from polls_question>
1. 좋아하는 색깔은 무엇입니까?
2. 좋아하는 동물은 무엇입니까?
3. 싫어하는 색깔은 고르세요.
4. 여행으로 가고 싶은 나라를 선택해주세요.
5. 배우고 싶은 언어는 무엇입니까?


In [77]:
result = Choice.objects.raw("select * from polls_choice")
for r in result:
    print(r, r.question)

1. 빨강 1. 좋아하는 색깔은 무엇입니까?
2. 파랑 1. 좋아하는 색깔은 무엇입니까?
3. 노랑 1. 좋아하는 색깔은 무엇입니까?
4. 보라 1. 좋아하는 색깔은 무엇입니까?
5. 빨강 3. 싫어하는 색깔은 고르세요.
6. 검정 3. 싫어하는 색깔은 고르세요.
7. 보라 3. 싫어하는 색깔은 고르세요.
8. 주황 3. 싫어하는 색깔은 고르세요.
9. 곰 2. 좋아하는 동물은 무엇입니까?


In [82]:
for q in Question.objects.all():
    print(q.pk, q.question_text)

1 좋아하는 색깔은 무엇입니까?
2 좋아하는 동물은 무엇입니까?
3 싫어하는 색깔은 고르세요.
4 여행으로 가고 싶은 나라를 선택해주세요.
5 배우고 싶은 언어는 무엇입니까?


In [92]:
q = Question.objects.get(pk=1)
print(q.pk, q.question_text, q.pub_date)
for c in q.choice_set.all():
    print(c.pk, c.choice_text)

1 좋아하는 색깔은 무엇입니까? 2025-07-07 05:17:02.657964+00:00
1 빨강
2 파랑
3 노랑
4 보라
